In [1]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env.yaml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})
os.environ['DATABASE_URL'] = 'sqlite:///:memory:'
os.environ['WSL_DISTRIBUTION'] = 'Ubuntu'

from hemlock import push_app_context

app = push_app_context()

In [2]:
[os.remove(t) for t in app.tmpfiles if os.path.exists(t)]

[]

In [3]:
from survey.utils import X_test, y_test, model, gen_profile_table, explainer

/home/dsbowen/algorithm-fairness/venv/lib/python3.6/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [4]:
import random

idx = random.choice(list(range(len(X_test))))
x, y = X_test.iloc[idx], y_test.iloc[idx]
x

male                1
age                22
juv_fel_count       0
juv_misd_count      1
juv_other_count     0
priors_count        3
felony              1
black               1
married             0
Name: 4119, dtype: int64

In [5]:
output = model.predict_proba(x.values)
output

array([0.6563937])

In [6]:
print(gen_profile_table(x, 'Offender profile'))

<table class="table table-hover">
    <thead>
        <tr>
            <th scope="col">Information about the criminal population in Broward County, Florida</th>
<th scope="col">Offender profile</th>
        </tr>
    </thead>
    <tbody>
        <tr><td>The average offender has 2 prior convictions</td>
<td><i>Number of prior convictions:</i> 3</td></tr>
<tr><td>The average offender is 29 years old</td>
<td><i>Age:</i> 22</td></tr>
<tr><td>66 of every 100 offenders' most recent crime was a felony</td>
<td><i>Most recent crime:</i> Felony</td></tr>
<tr><td>55 of every 100 offenders are Black</td>
<td><i>Race:</i> Black</td></tr>
<tr><td>81 of every 100 offenders are male</td>
<td><i>Sex:</i> Male</td></tr>
<tr><td>The average offender has 0 juvenile felonies, 0 juvenile misdemeanors, and 0 other juvenile offenses</td>
<td>This offender has 0 juvenile felonies, 1 juvenile misdemeanors, and 0 other juvenile offenses</td></tr>
<tr><td>9 of every 100 offenders are married</td>
<td><i>Marital

In [7]:
explainer.explain_observations(x)

['<p>The average offender has a 52 in 100 chance of committing another crime within 2 years.</p><p>The most important factor when considering this specific offender is age. Younger offenders are more likely to commit future crimes. This offender is <b>much younger</b> than most offenders. This <b>increases</b> the chances the offender will commit another crime to 62 in 100.</p>\n<p>Additionally, offenders with more juvenile offenses are more likely to commit future crimes. This offender has <b>more juvenile offenses</b> than most offenders. This <b>increases</b> the chances the offender will commit another crime to 66 in 100.</p>\n            <p>After making small adjustments for other factors, my final estimate is that there is a <b>66 in 100</b> chance the offender will commit another crime within 2 years.</p>\n            ']

In [8]:
round(100*output[0])

66

In [10]:
import pandas as pd

x = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
x

a    1
b    2
c    3
dtype: int64

In [13]:
x = pd.Series(
    [0, 26, 1, 0, 1, 0, 0, 0, 0],
    index=[
        'priors_count',
        'age',
        'felony',
        'black',
        'male',
        'juv_fel_count',
        'juv_misd_count',
        'juv_other_count',
        'married'
    ]
)
x.to_frame().T

,priors_count,age,felony,black,male,juv_fel_count,juv_misd_count,juv_other_count,married
0,0,26,1,0,1,0,0,0,0


In [14]:
tmp = x.to_frame().T[X_test.columns]
tmp

,male,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,felony,black,married
0,1,26,0,0,0,0,1,0,0


In [15]:
x.astype(int)

priors_count        0
age                26
felony              1
black               0
male                1
juv_fel_count       0
juv_misd_count      0
juv_other_count     0
married             0
dtype: int64

In [16]:
x = x[X_test.columns]
x

male                1
age                26
juv_fel_count       0
juv_misd_count      0
juv_other_count     0
priors_count        0
felony              1
black               0
married             0
dtype: int64